In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Lambda-School-Labs/Labs26-Citrics-DS-TeamA/master/data/rental/apt_rental_price_clean1.csv"

df = pd.read_csv(url)
print(df.shape)
df.head()

(2220, 83)


,city,state,bedroom_size,price_2014_01,price_2014_02,price_2014_03,price_2014_04,price_2014_05,price_2014_06,price_2014_07,price_2014_08,price_2014_09,price_2014_10,price_2014_11,price_2014_12,price_2015_01,price_2015_02,price_2015_03,price_2015_04,price_2015_05,price_2015_06,price_2015_07,price_2015_08,price_2015_09,price_2015_10,price_2015_11,price_2015_12,price_2016_01,price_2016_02,price_2016_03,price_2016_04,price_2016_05,price_2016_06,price_2016_07,price_2016_08,price_2016_09,price_2016_10,price_2016_11,price_2016_12,price_2017_01,...,price_2017_05,price_2017_06,price_2017_07,price_2017_08,price_2017_09,price_2017_10,price_2017_11,price_2017_12,price_2018_01,price_2018_02,price_2018_03,price_2018_04,price_2018_05,price_2018_06,price_2018_07,price_2018_08,price_2018_09,price_2018_10,price_2018_11,price_2018_12,price_2019_01,price_2019_02,price_2019_03,price_2019_04,price_2019_05,price_2019_06,price_2019_07,price_2019_08,price_2019_09,price_2019_10,price_2019_11,price_2019_12,price_2020_01,price_2020_02,price_2020_03,price_2020_04,price_2020_05,price_2020_06,price_2020_07,price_2020_08
0,New York,NY,Studio,1780,1778,1781,1786,1794,1802,1817,1824,1828,1828,1829,1827,1822,1822,1825,1831,1838,1846,1859,1866,1867,1863,1860,1853,1846,1841,1839,1843,1847,1849,1856,1865,1885,1885,1875,1866,1857,...,1865,1872,1880,1883,1882,1879,1872,1868,1863,1860,1861,1867,1883,1905,1899,1897,1894,1908,1915,1919,1907,1897,1889,1904,1920,1936,1933,1928,1922,1937,1940,1943,1939,1938,1939,1934,1923,1904,1886,1864
1,New York,NY,1br,1976,1975,1978,1983,1992,2001,2018,2026,2030,2030,2031,2029,2023,2023,2027,2034,2041,2050,2065,2072,2073,2069,2065,2058,2050,2044,2042,2047,2051,2054,2061,2070,2093,2093,2082,2072,2062,...,2071,2079,2088,2091,2090,2086,2079,2075,2069,2065,2067,2073,2091,2116,2109,2106,2104,2119,2126,2131,2118,2106,2098,2114,2132,2150,2146,2142,2134,2151,2155,2158,2153,2152,2153,2148,2135,2115,2094,2070
2,New York,NY,2br,2354,2352,2356,2362,2373,2384,2404,2413,2418,2419,2419,2417,2410,2410,2415,2423,2432,2442,2459,2469,2470,2465,2460,2452,2442,2435,2433,2439,2443,2447,2455,2466,2494,2493,2481,2468,2456,...,2467,2477,2487,2491,2489,2485,2477,2472,2465,2460,2462,2469,2491,2521,2513,2509,2506,2524,2533,2538,2523,2509,2499,2518,2540,2561,2557,2551,2542,2562,2567,2570,2565,2564,2565,2559,2543,2519,2494,2466
3,New York,NY,3br,3036,3033,3038,3046,3060,3074,3099,3112,3118,3119,3119,3116,3108,3108,3114,3124,3136,3149,3171,3183,3185,3178,3172,3161,3149,3140,3137,3144,3150,3155,3166,3180,3215,3214,3199,3182,3167,...,3181,3194,3207,3212,3210,3205,3194,3187,3178,3172,3174,3184,3212,3250,3240,3235,3231,3255,3266,3273,3253,3235,3222,3247,3275,3302,3297,3289,3278,3304,3310,3314,3307,3306,3308,3300,3279,3248,3216,3180
4,New York,NY,4br,3381,3378,3383,3392,3407,3424,3452,3466,3472,3473,3474,3471,3461,3461,3468,3479,3492,3507,3532,3545,3547,3540,3533,3521,3507,3497,3494,3502,3508,3513,3526,3542,3581,3580,3562,3544,3527,...,3542,3557,3571,3578,3575,3569,3557,3549,3540,3533,3535,3546,3577,3620,3608,3603,3599,3625,3638,3645,3623,3603,3588,3616,3648,3677,3672,3663,3651,3680,3686,3691,3684,3682,3684,3675,3652,3617,3582,3541


In [2]:
import plotly.express as px

subset = df[df.city == "Atlanta"]
fig = px.line(
    subset, 
    x="bedroom_size", 
    y="price_2020_08",
    title="Rental Price Estimates for Atlanta, GA"
)
fig.show()

In [5]:
def rentalviz(city):
    subset = df[df.city == city]
    fig = px.bar(
        subset,
        x="bedroom_size",
        y="price_2020_08",
        title=f"Rental Price Estimates for {city}"
    )
    fig.show()
    return fig.to_json()

In [6]:
rentalviz("Baltimore")

'{"data":[{"alignmentgroup":"True","hoverlabel":{"namelength":0},"hovertemplate":"bedroom_size=%{x}<br>price_2020_08=%{y}","legendgroup":"","marker":{"color":"#636efa"},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"textposition":"auto","type":"bar","x":["Studio","1br","2br","3br","4br"],"xaxis":"x","y":[784,952,1194,1535,1798],"yaxis":"y"}],"layout":{"barmode":"relative","legend":{"tracegroupgap":0},"template":{"data":{"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"bar"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"barpolar"}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"choropleth":[{"colorbar":{"outlinewidth":0,"ticks":""},"ty

In [7]:
rentalviz("Atlanta")

'{"data":[{"alignmentgroup":"True","hoverlabel":{"namelength":0},"hovertemplate":"bedroom_size=%{x}<br>price_2020_08=%{y}","legendgroup":"","marker":{"color":"#636efa"},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"textposition":"auto","type":"bar","x":["Studio","1br","2br","3br","4br"],"xaxis":"x","y":[979,1027,1187,1558,1914],"yaxis":"y"}],"layout":{"barmode":"relative","legend":{"tracegroupgap":0},"template":{"data":{"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"bar"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"barpolar"}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"choropleth":[{"colorbar":{"outlinewidth":0,"ticks":""},"t

In [10]:
import os
import psycopg2
import pandas as pd




def fetch_query(query, columns):
    """
    Creates a connection to database, returns query from specified table
    as a list of dictionaries.

    Input: query: a SQL query (string)
    """
    # DB_NAME = os.getenv("DB_NAME")
    # DB_USER = os.getenv("DB_USER")
    # DB_PASSWORD = os.getenv("DB_PASSWORD")
    # DB_HOST = os.getenv("DB_HOST")

    DB_USER = "citrics"
    DB_PASSWORD = "BnDW2WupbFpgZSewsZm7"
    DB_NAME = "postgres"
    DB_HOST = "citricsads.cav8gkdxva9e.us-east-1.rds.amazonaws.com"

    # Creating Connection Object
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST)
    # Creating Cursor Object
    cursor = conn.cursor()
    # Fetch comments query
    query = query
    # Execute query
    cursor.execute(query)
    # Query results
    comments = list(cursor.fetchall())
    # Key-value pair names for df columns
    columns = columns
    # List of tuples to DF
    df = pd.DataFrame(comments, columns=columns)
    print(type(df))
    # DF to dictionary
    pairs = df.to_json(orient='records')
    print(type(pairs))
    # Closing Connection
    conn.close()

    return pairs


In [47]:
query = """
    SELECT
        city,
        "state",
        bedroom_size,
        price_2020_08
    FROM rp_clean1
    WHERE bedroom_size = '1br' OR bedroom_size = 'Studio'
    """

columns = ["city", "state", "bedroom_size", "price_2020_08"]

df = pd.read_json(fetch_query(query, columns))

<class 'pandas.core.frame.DataFrame'>
<class 'str'>


In [119]:
# !pip install fastapi

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 8.7MB 9.6MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 


In [122]:
from fastapi import HTTPException

def rentalviz(cityname, x):
  """

  """
  # Sanitize input string (cityname param)
  cityname = cityname.lower()
  cityname = smart_upper(cityname)

  # Make a set of citynames in the rental price data
  citynames = set(x.city.to_list())

  if cityname not in citynames:
    raise HTTPException(
        status_code=404, 
        detail=f'City name "{cityname}" not found!'
        )
  
  # Extract statecode (for use in title)
  statecode = x[x.city == cityname]["state"].to_list()[0]

  subset = x[x.city == cityname]
  fig = px.bar(
      subset,
      x="bedroom_size",
      y="price_2020_08",
      title=f"Rental Price Estimates for {cityname}, {statecode}"
  )

  fig.show()
  return fig.to_json()

In [50]:
df.head()

,city,state,bedroom_size,price_2020_08
0,New York,NY,Studio,1864
1,New York,NY,1br,2070
2,Los Angeles,CA,Studio,1138
3,Los Angeles,CA,1br,1354
4,Chicago,IL,Studio,939


In [49]:
x = [v.lower() for v in df.city.to_list()]

copy = df.copy()

copy.city = x

copy.head()

,city,state,bedroom_size,price_2020_08
0,new york,NY,Studio,1864
1,new york,NY,1br,2070
2,los angeles,CA,Studio,1138
3,los angeles,CA,1br,1354
4,chicago,IL,Studio,939


In [55]:
v = df.city.to_list()
v[0]

'New York'

In [92]:
this = x[0][0].upper() + x[0][1:]
[this]

['New york']

In [93]:
" ".join(this)

'N e w   y o r k'

In [66]:
x[0][1:]

'ew york'

In [85]:
a = x[1].split()
print(len(a))
a

2


['new', 'york']

In [86]:
a[0][0]

'n'

In [87]:
" ".join(a)

'new york'

In [111]:
string = "St. Louis"
string.split()

['St.', 'Louis']

In [112]:
def smart_upper(x):
  """"
  Upper case the first letter of a string, as well as
  any letters directly following a space.

  Input: x: lowercase str (e.g. 'new york')

  Returns: new_string: str (e.g. 'New York')
  """

  # Can start by splitting the string
  split_str = x.split()

  if len(split_str) == 1:

    first_letter = split_str[0][0]

    # If no space was in string, simply uppercase the first letter
    first_letter = first_letter.upper()
    new_string = first_letter + x[1:]

    return new_string

  wb = []
  for v in split_str:
    v = smart_upper(v)
    wb.append(v)

  # And uppercase the first letter of each string
  return " ".join(wb)

In [115]:
my_string = "let's get reaaaaly fancy with this, and see what happens!"
smart_upper(my_string)

"Let's Get Reaaaaly Fancy With This, And See What Happens!"

In [110]:
df.loc[df.city == "Washington"]

,city,state,bedroom_size,price_2020_08
50,Washington,DC,Studio,1273
51,Washington,DC,1br,1337


### Re-define `rentalviz` by packaging the two functions together

In [131]:
from fastapi import HTTPException

def rentalviz(cityname, x):
  """

  """
  # Function to sanitize input string (cityname param)
  def smart_upper(x):
    """"
    Upper case the first letter of a string, as well as
    any letters directly following a space.

    Input: x: lowercase str (e.g. 'new york')

    Returns: new_string: str (e.g. 'New York')
    """

    # Can start by splitting the string
    split_str = x.split()

    if len(split_str) == 1:

      first_letter = split_str[0][0]

      # If no space was in string, simply uppercase the first letter
      first_letter = first_letter.upper()
      new_string = first_letter + x[1:]

      return new_string

    # Empty array for "word basket"
    wb = []
    # Iterate through words in `split_str`
    for v in split_str:
      # Recursive `smart_upper` call for each word
      v = smart_upper(v)
      wb.append(v)

    # Join smart-uppercased words in word basket
    return " ".join(wb)

  # Input sanitization
  cityname = cityname.lower()
  cityname = smart_upper(cityname)

  # Make a set of citynames in the rental price data
  citynames = set(x.city.to_list())

  # Raise HTTPException for unknown inputs
  if cityname not in citynames:
    raise HTTPException(
        status_code=404, 
        detail=f'City name "{cityname}" not found!'
        )
  
  # Otherwise, extract statecode (for use in title)
  statecode = x[x.city == cityname]["state"].to_list()[0]

  # Get subset for cityname input
  subset = x[x.city == cityname]

  # Create visualization
  fig = px.bar(
      subset,
      x="bedroom_size",
      y="price_2020_08",
      title=f"Rental Price Estimates for {cityname}, {statecode}"
  )

  fig.show()
  return fig.to_json()

In [132]:
rentalviz("cHIcaGO", df)

'{"data":[{"alignmentgroup":"True","hoverlabel":{"namelength":0},"hovertemplate":"bedroom_size=%{x}<br>price_2020_08=%{y}","legendgroup":"","marker":{"color":"#636efa"},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"textposition":"auto","type":"bar","x":["Studio","1br"],"xaxis":"x","y":[939,1090],"yaxis":"y"}],"layout":{"barmode":"relative","legend":{"tracegroupgap":0},"template":{"data":{"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"bar"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"barpolar"}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"choropleth":[{"colorbar":{"outlinewidth":0,"ticks":""},"type":"choropleth"}],"contour":[{"

In [12]:
print(df.shape)
df.head()

(888, 4)


,city,state,bedroom_size,price_2020_08
0,New York,NY,Studio,1864
1,New York,NY,1br,2070
2,Los Angeles,CA,Studio,1138
3,Los Angeles,CA,1br,1354
4,Chicago,IL,Studio,939


In [24]:
rentalviz("Nashville")

'{"data":[{"alignmentgroup":"True","hoverlabel":{"namelength":0},"hovertemplate":"bedroom_size=%{x}<br>price_2020_08=%{y}","legendgroup":"","marker":{"color":"#636efa"},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"textposition":"auto","type":"bar","x":["Studio","1br"],"xaxis":"x","y":[851,948],"yaxis":"y"}],"layout":{"barmode":"relative","legend":{"tracegroupgap":0},"template":{"data":{"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"bar"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5}},"type":"barpolar"}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"choropleth":[{"colorbar":{"outlinewidth":0,"ticks":""},"type":"choropleth"}],"contour":[{"c